In [16]:
import scipy.io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize

from scipy.io import loadmat, savemat
import pandas as pd
import scipy.special as SS
import scipy.stats as SSA
import copy
import random
import math
from sklearn.model_selection import ParameterGrid
import os
import numpy.linalg as LA
import gzip

# load pickle module
import pickle
import geopandas as gpd
from matplotlib.colors import Normalize
import matplotlib.colors as mcolors

In [2]:
usa = gpd.read_file('/Users/qingyao/OneDrive - cumc.columbia.edu/machine_learning_data/cb_2020_us_county_500k/cb_2020_us_county_500k.shp')
usa_main = usa[~usa['STATEFP'].isin(["02","15","60","66","69","71","72","78"])]
usa_main['FIPS'] = usa_main['GEOID'].astype('int64')
usa_main_sorted = usa_main.sort_values(by='FIPS')

/Users/qingyao/anaconda3/envs/geo/lib/python3.11/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [43]:
usa_main_projected = usa_main_sorted.to_crs(epsg=5070)
usa_main_projected['centroids'] = usa_main_projected.geometry.centroid
usa_main_projected['centroid_x'] = usa_main_projected['centroids'].x
usa_main_projected['centroid_y'] = usa_main_projected['centroids'].y

In [55]:
result_dir = '/Users/qingyao/Documents/branching_data/simulation/with_mobility/'

In [56]:
import pickle

# Load from a file
with open(result_dir+'test.pkl', 'rb') as file:
    loaded_data = pickle.load(file)
print(loaded_data.keys())

dict_keys(['seeds', 'rs', 'M_E_NewInf'])


In [57]:
simulated_cases =  loaded_data['M_E_NewInf'][0]

In [58]:
dates_idx = pd.date_range('2020-02-21',periods=60)
formatted_dates = dates_idx.strftime('%m-%d-%Y').tolist()
s_cases = pd.DataFrame(index=range(3108), columns = formatted_dates)
s_cases.iloc[:,:] = simulated_cases

In [59]:
df_fips_sorted = pd.read_csv('fips_sorted.cvs')
s_cases['FIPS'] = df_fips_sorted['FIPS']

In [60]:
df_cases_us_geo_projected = usa_main_projected.merge(s_cases,how='left',left_on='FIPS',right_on='FIPS')

In [41]:
def find_the_large_link(arr, rank):
    arr_copy = arr.copy()
    
    # Use a large negative value for the diagonal.
    np.fill_diagonal(arr_copy, -np.iinfo(np.int32).max)
    
    sorted_indices = np.argsort(arr_copy, axis=None)
    top_10_indices = sorted_indices[-rank:]
    row_indices, col_indices = np.unravel_index(top_10_indices, arr.shape)
    
    return list(zip(row_indices, col_indices))

In [62]:
# dates_idx = pd.date_range('2020-02-21',periods=60)
# formatted_dates = dates.strftime('%Y-%m-%d').tolist()
file_path = '/Users/qingyao/Documents/branching_data/real_data/'
norm = Normalize(vmin=0, vmax=100)
###########################
dates = formatted_dates[:60]
########################## 
colors = plt.cm.YlOrRd(np.linspace(0, 1, 128))
colors_with_white = np.vstack(([1, 1, 1, 1], colors))  # Add white at the start
custom_colormap = mcolors. LinearSegmentedColormap.from_list("custom_YlOrRd", colors_with_white)

for i, date in enumerate(dates):
#     safe_date = date
#     print(safe_date)
    safe_date = date.replace("-", "_")  # Replace / with _
    fig, ax = plt.subplots(1 , figsize=(10, 6), dpi=450)
    
    plot_obj = df_cases_us_geo_projected.plot(column=date, cmap=custom_colormap, 
                                              linewidth=0.02, ax=ax, edgecolor='0.1',
                                              norm=norm)
    
    ax.axis('off')
    ax.set_title('daily cases for ' + date, fontdict={'fontsize': '12', 'fontweight' : '3'})
    
    cax = fig.add_axes([0.9, 0.2, 0.01, 0.5])  
    cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap='YlOrRd'), cax=cax)
    cbar.set_label('Number of Cases per 100K pop')
#     plt.show()

    
    m, d, y = safe_date.split('_')
    M_test = np.loadtxt(file_path+'M_{}{}_{}.csv'.format(y,m,d),delimiter=',')

    large_links = find_the_large_link(M_test,2000)
    arr_normalized = 5*(M_test - M_test.min()) / (M_test.max() - M_test.min())
    
    for (in_county, out_county) in large_links:      
#         print(out_county, df_cases_us_geo_projected.iloc[out_county]['FIPS'])
#         print(in_county, df_cases_us_geo_projected.iloc[in_county]['FIPS'])
         
        out_x = df_cases_us_geo_projected.iloc[out_county]['centroid_x']
        out_y = df_cases_us_geo_projected.iloc[out_county]['centroid_y']

        in_x = df_cases_us_geo_projected.iloc[in_county]['centroid_x']
        in_y = df_cases_us_geo_projected.iloc[in_county]['centroid_y']
        
        arrowprops = dict(arrowstyle='->,head_width=0.1,head_length=0.1', 
                          linewidth=arr_normalized[in_county, out_county], 
                          color='blue', connectionstyle="arc3,rad=0.3")
        
        ax.annotate('', xy=(in_x, in_y), xytext=(out_x, out_y),
                    arrowprops=arrowprops) 
#         ax.arrow(out_x, out_y,  
#                  in_x - out_x, in_y - out_y, 
         
#                  head_width=0.1, head_length=0.5, fc='blue', ec='blue',linewidth=arr_normalized[in_county, out_county])
    
    save_dir = '/Users/qingyao/Documents/branching_data/maps/simulate/'
    
    fig.savefig(save_dir+"plots/map_{}.png".format(safe_date),bbox_inches='tight', pad_inches=0)
    plt.close(fig)